In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

import pandas as pd
import numpy as np

import time
import datetime

import os
import requests

In [8]:
def main():
    df = pd.DataFrame(columns=['Job_Title', 'Company', 'City', 'Salary', 'Description', 'Experience', 'Education', 'Skills'])
    driver = webdriver.Chrome()
    url = 'https://www.naukri.com/data-scientist-jobs'
    driver.get(url)
    time.sleep(10)

    try:
        # Handle popups or filters if any
        driver.find_element(By.XPATH, '//*[@id="root"]/div[4]/div[1]/div/section[2]/div[1]/div[2]/span/span[2]/p').click()
        driver.find_element(By.XPATH, '//*[@id="root"]/div[4]/div[1]/div/section[2]/div[1]/div[2]/span/span[2]/ul/li[2]').click()
    except Exception as e:
        pass

    pages = np.arange(1, 11) # scrap first 10 pages
    for page in pages:
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        results = soup.find(class_="styles_job-listing-container__OCfZC")

        if results:
            result2 = results.find_all('div', class_='srp-jobtuple-wrapper')

            for job_elem in result2:
                # Scrape Job Title
                T = job_elem.find('a', class_="title ")
                Title = T.text if T else "Not Mentioned"

                # Scrape company name
                company_name = "Not Mentioned"
                try:
                    C1 = job_elem.find('a', class_="comp-name mw-25")
                    Company = C1.text if C1 else "Not Mentioned"
                except Exception as e:
                    pass  # No company found in the first link

                # Try to scrape company from the second link
                try:
                    C2 = job_elem.find('a', class_=" comp-dtls-wrap")
                    if C2 and C2.text:
                        Company = C2.text
                except Exception as e:
                    pass  # No company found in the second link
                
                # Scrape City
                C = job_elem.find('span', class_="ni-job-tuple-icon ni-job-tuple-icon-srp-location loc")
                City = C.text if C else "Not Mentioned"

                # Scrape Salary
                S = job_elem.find('span', class_='ni-job-tuple-icon ni-job-tuple-icon-srp-rupee sal')
                Salary = S.text if S else "Not Mentioned"

                # Scrape Description
                D = job_elem.find('span', class_="job-desc ni-job-tuple-icon ni-job-tuple-icon-srp-description")
                Description = D.text if D else "Not Mentioned"

                # Scrape Experience
                E = job_elem.find('span', class_="ni-job-tuple-icon ni-job-tuple-icon-srp-experience exp")
                Exp = E.text if E else "Not Mentioned"

                # Click on the job title to open the detailed page
                job_link = job_elem.find('a').get('href')
                driver.execute_script("window.open(arguments[0])", job_link)
                driver.switch_to.window(driver.window_handles[-1])  # Switch to the new tab
                time.sleep(5)  # Wait for the job details page to load

                # Scrape Education and Skills from detailed job page
                try:
                    Education = driver.find_element(By.XPATH, '//*[@id="root"]/div/main/div[1]/div[1]/section[2]/div[2]/div[3]').text
                except:
                    Education = "Not Mentioned"
                    
                skills_list = []
        
                try:
                    skill_elements_1 = driver.find_elements(By.XPATH, '//*[@id="root"]/div/main/div[1]/div[1]/section[2]/div[3]/div[2]/a')
    
                # Loop through each element to extract text
                    for skill_element in skill_elements_1:
                        skills_list.append(skill_element.text)

                except Exception as e:
                    pass
                
                try:
                    skill_elements_2 = driver.find_elements(By.XPATH, '//*[@id="root"]/div/main/div[1]/div[1]/section[2]/div[3]/div[4]/a')
                    for skill_element in skill_elements_2:
                        skills_list.append(skill_element.text)
                except Exception as e:
                    pass  # No skills found in the second section

                if not skills_list:
                    skills_list.append("Not Mentioned")  # If no skills were found in either section


                # Close the job detail tab and switch back to the main page
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                # Append data to DataFrame
                df2 = pd.DataFrame([[Title, Company, City, Salary, Description, Exp, Education, skills_list]],
                                    columns=['Job_Title', 'Company', 'City', 'Salary', 'Description', 'Experience', 'Education', 'Skills'])
                df = pd.concat([df, df2], ignore_index=True)

        # Go to the next page
        try:
            next_button = driver.find_element(By.XPATH, '//*[@id="lastCompMark"]/a[2]').get_attribute('href')
            driver.get(next_button)
            time.sleep(5)  # Give time for the page to load
        except Exception as e:
            print(f"Error navigating to next page: {e}")
            break
                 
    # Save to Excel
    output_file = "data-science-skills.xlsx"
    df.to_excel(output_file, index=False, engine='openpyxl')
    print(f"Data saved to {output_file}")

main()

Data saved to data-science-skills.xlsx
